<a href="https://colab.research.google.com/github/gigaanderson/ETL/blob/main/Teste_Qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Código Python para testar  a busaca por similaridade no Banco de Dados Vetorial QDRANT, usando como base um vetor de dimensão 128 a distância de Cosine.

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams
import numpy as np

In [ ]:
#Serviço free online
qdrant_client = QdrantClient(
    url="https://e434e6fc-4123-4de5-88a5-aaf83bb0e12c.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="KCGlUpoAescNNPpVrVG_fgaIalmYDiDhWk71Z7aVzUSmGqTuGIK44",
    check_compatibility=False,
)

In [ ]:
# Definir o nome da coleção
collection_name = "teste"

# Criar uma coleção com vetores de dimensão 128
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=128, distance="Cosine")
)

# Gerar alguns vetores aleatórios
vectors = np.random.rand(10, 128)  # 10 vetores de dimensão 128
ids = list(range(1, 11))  # IDs para os vetores

# Inserir os vetores na coleção
qdrant_client.upsert(
    collection_name=collection_name,
    points=[
        {"id": id_, "vector": vector.tolist()}
        for id_, vector in zip(ids, vectors)
    ]
)

# Vetor de consulta (um vetor qualquer de dimensão 128)
query_vector = np.random.rand(128)

# Buscar os 5 vetores mais similares ao vetor de consulta
search_result = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_vector.tolist(),
    limit=5
)

# Exibir resultados da busca
for result in search_result:
    print(f"ID: {result.id}, Distância: {result.score}")


print(qdrant_client.get_collections())

In [ ]:
# Nome da coleção
collection_name = "senhas_colecao"

# Criar coleção no Qdrant (com vetores de dimensão 16 para armazenar os tamanhos das senhas)
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=16, distance="Cosine")
)

# Função para ler o arquivo senhas.txt
def ler_senhas(arquivo):
    senhas = []
    with open(arquivo, "r") as f:
        for linha in f:
            partes = linha.strip().split()
            if len(partes) >= 3:
                tamanho = int(partes[0])
                ocorrencias = int(partes[1])
                senha = ' '.join(partes[2:])  # Caso a senha contenha espaços
                senhas.append({"tamanho": tamanho, "ocorrencias": ocorrencias, "senha": senha})
    return senhas

# Lendo o arquivo 'senhas.txt'
arquivo = "senhas.txt"  # Coloque o caminho correto do arquivo aqui
senhas = ler_senhas(arquivo)

# Convertendo as senhas para vetores e inserindo no Qdrant
for idx, senha in enumerate(senhas):
    # Criação de um vetor aleatório para cada senha (neste exemplo)
    vetor = np.random.rand(16)  # Substitua pelo vetor desejado, se aplicável
    qdrant_client.upsert(
        collection_name=collection_name,
        points=[
            {
                "id": idx + 1,
                "vector": vetor.tolist(),
                "payload": {
                    "tamanho": senha["tamanho"],
                    "ocorrencias": senha["ocorrencias"],
                    "senha": senha["senha"]
                }
            }
        ]
    )

print("Impressão completa.")


print(qdrant_client.get_collections())